# Load Database with state and energy data

## Import libraries

In [1]:
# Dependencies
import pandas as pd
import os
import sqlite3

In [2]:
# Import state coordinates
state_path = os.path.join("static", "data", "state_coordinates", "statelatlong.csv")

In [3]:
# Read our Data file with the pandas library
state_coordinates_df = pd.read_csv(state_path, encoding="ISO-8859-1")

In [4]:
# Show just the header
state_coordinates_df.head()

,State,Latitude,Longitude,City
0,US,37.090200,-95.712900,United States
1,AL,32.601011,-86.680736,Alabama
2,AK,61.302501,-158.775020,Alaska
3,AZ,34.168219,-111.930907,Arizona
4,AR,34.751928,-92.131378,Arkansas


In [5]:
# rename columns
state_coordinates_df = state_coordinates_df.rename(columns={'State': 'Abbreviation', 'City': 'US_State'})
state_coordinates_df.head()

,Abbreviation,Latitude,Longitude,US_State
0,US,37.090200,-95.712900,United States
1,AL,32.601011,-86.680736,Alabama
2,AK,61.302501,-158.775020,Alaska
3,AZ,34.168219,-111.930907,Arizona
4,AR,34.751928,-92.131378,Arkansas


In [6]:
# Export state coordinates data to csv file
state_coordinates_path = os.path.join("static", "data", "state_coordinates", "state_coordinates.csv")
state_coordinates_df.to_csv(state_coordinates_path, index=False, header=True)

## Store data in a SQLite database

In [7]:
# Create connection to the database
database_path = os.path.join("static", "db", "us_energy.sqlite")
conn = sqlite3.connect(database_path)

In [8]:
# Store state coordinates data in a table
state_coordinates_df.to_sql('state_coordinates', conn, if_exists='replace', index=False)

c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE state_coordinates RENAME TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE state_coordinates (Abbreviation TEXT PRIMARY KEY NOT NULL,
                                    Latitude NUMERIC,
                                    Longitude NUMERIC,
                                    US_State TEXT);

    INSERT INTO state_coordinates SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()

In [9]:
# Read the state coordinates table
pd.read_sql('select * from state_coordinates', conn).head()

,Abbreviation,Latitude,Longitude,US_State
0,US,37.090200,-95.712900,United States
1,AL,32.601011,-86.680736,Alabama
2,AK,61.302501,-158.775020,Alaska
3,AZ,34.168219,-111.930907,Arizona
4,AR,34.751928,-92.131378,Arkansas


In [10]:
# Import combined data from csv file
final_path = os.path.join("static", "data", "EIA_Project_Data", "final_combine_table.csv")

In [11]:
# Read our Data file with the pandas library
final_combine_table_df = pd.read_csv(final_path, encoding="ISO-8859-1")

In [12]:
# Show just the header
final_combine_table_df.head()

,State,Year,Total_co2_emission,CO2_Unit,Average_Price,Price_Unit,resident_population,Pop_Unit,Total_energy,ENERGY_Unit,Total_renewable_energy,Renew_Unit
0,US,2016,5160.990850,million metric tons CO2,15.94,Dollars per million Btu,323071,Thousand,84247690,Billion Btu,8053314,Billion Btu
1,US,2015,5207.386647,million metric tons CO2,17.30,Dollars per million Btu,320743,Thousand,88197690,Billion Btu,7488974,Billion Btu
2,US,2014,5350.265440,million metric tons CO2,21.33,Dollars per million Btu,318386,Thousand,87613590,Billion Btu,7670927,Billion Btu
3,US,2013,5301.451176,million metric tons CO2,21.42,Dollars per million Btu,316058,Thousand,81705691,Billion Btu,7445307,Billion Btu
4,US,2012,5162.563034,million metric tons CO2,21.83,Dollars per million Btu,313874,Thousand,79131447,Billion Btu,6936925,Billion Btu


In [13]:
# Store energy data in a table
final_combine_table_df.to_sql('final_combine_table', conn, if_exists='replace', index=False)

c = conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE final_combine_table RENAME TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE final_combine_table (State TEXT NOT NULL,
                                    Year INTEGER NOT NULL,
                                    Total_co2_emission NUMERIC,
                                    CO2_Unit TEXT,
                                    Average_Price NUMERIC,
                                    Price_Unit TEXT,
                                    Average_resident_population NUMERIC,
                                    Pop_Unit TEXT,
                                    Total_energy NUMERIC,
                                    ENERGY_Unit TEXT,
                                    Total_renewable_energy NUMERIC,
                                    Renew_Unit TEXT,
                                     PRIMARY KEY (State, Year));

    INSERT INTO final_combine_table SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()

In [14]:
# Read the energy data table
pd.read_sql('select * from final_combine_table', conn).head()

,State,Year,Total_co2_emission,CO2_Unit,Average_Price,Price_Unit,Average_resident_population,Pop_Unit,Total_energy,ENERGY_Unit,Total_renewable_energy,Renew_Unit
0,US,2016,5160.990850,million metric tons CO2,15.94,Dollars per million Btu,323071,Thousand,84247690,Billion Btu,8053314,Billion Btu
1,US,2015,5207.386647,million metric tons CO2,17.30,Dollars per million Btu,320743,Thousand,88197690,Billion Btu,7488974,Billion Btu
2,US,2014,5350.265440,million metric tons CO2,21.33,Dollars per million Btu,318386,Thousand,87613590,Billion Btu,7670927,Billion Btu
3,US,2013,5301.451176,million metric tons CO2,21.42,Dollars per million Btu,316058,Thousand,81705691,Billion Btu,7445307,Billion Btu
4,US,2012,5162.563034,million metric tons CO2,21.83,Dollars per million Btu,313874,Thousand,79131447,Billion Btu,6936925,Billion Btu
